In [ ]:
# Cell 1: Install dependencies (latest stable versions for Colab)
!pip install -q torch torchvision torchaudio
!pip install -q transformers sentence-transformers faiss-cpu flask pyngrok


In [ ]:
# Cell 2: Import libraries and verify setup

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import faiss
from flask import Flask, request, jsonify
from pyngrok import ngrok
import yaml
import json

# Quick environment check
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Transformers, FAISS, Flask, pyngrok all imported successfully")


Torch version: 2.8.0+cu126
CUDA available: True
Transformers, FAISS, Flask, pyngrok all imported successfully


In [ ]:
# Cell 3
KNOWLEDGE_BASE = [
    {
        "title": " Shoplite User Registration Process",
        "content": "To create a Shoplite accounts, users must visit the registration page and provide their valid email or phone number, create a strong password, enter basic profile information.  To ensure security and prevent fraudulent accounts, an email verification is required within 24 hours of registration.If the verification process is not completed, the account remains inactive until confirmed.Shoplite offers two primary account types: **buyer accounts** and **seller accounts**. Buyer accounts are free to create and allow customers to browse products, add items to their cart, track orders, and leave product reviews. Seller accounts, on the other hand, require additional steps such as business verification. Sellers must provide tax identification numbers, proof of business registration, and contact details for verification, a process that typically takes two to three business days. Once approved, sellers gain access to a dedicated dashboard for managing inventory, processing orders, and tracking performance metrics.Account management tools are designed to give users control over their information. Buyers and sellers can update profile details, change passwords, and adjust notification preferences at any time. For enhanced security, Shoplite supports optional two-factor authentication (2FA), which can be enabled under account settings. In cases where users forget their password, a reset link is sent via email after identity verification.Shoplite also provides privacy controls, allowing users to manage how their data is shared with third parties. Account deactivation and permanent deletion are available options, ensuring compliance with global privacy standards. Overall, the registration and account management process on Shoplite balances ease of use with strong safeguards to protect both buyers and sellers.",
        "id": "doc1",
    },
    {
        "title": "Shoplite Product Search and Filtering Features",
        "content": "Efficient product discovery is a core feature of Shoplite, designed to help users quickly find the items they need. The platform provides a robust search bar that supports both keyword and natural language queries. For example, a buyer can search “wireless headphones under $100” and immediately receive tailored results. Shoplite also integrates category browsing, allowing users to navigate through structured sections such as electronics, fashion, home goods, and health products.To refine results, Shoplite offers a wide range of filtering options. Users can filter products by **price range, brand, customer rating, seller type (individual or verified business), and estimated delivery time**. Filters can be combined to narrow down choices, ensuring customers see the most relevant products. Sorting features further improve usability, with options such as “price: low to high,” “newest arrivals,” and “best reviewed.”Personalization is another strength of Shoplite’s search system. Returning users benefit from **saved searches and browsing history**, which allow for quick access to frequently viewed categories. Additionally, the platform uses recommendation algorithms to suggest products based on prior activity, trending items, and seasonal promotions. These recommendations appear in both search results and on the homepage.For mobile app users, voice search is supported, making product discovery even more accessible. The app can interpret voice commands and apply filters automatically, reducing the time needed to locate products. Advanced search capabilities, combined with intuitive filters and recommendations, ensure that buyers can efficiently navigate Shoplite’s extensive catalog while finding exactly what they’re looking for.",
        "id": "doc2",
    },
    {
        "title": "Shoplite Shopping Cart and Checkout Process",
        "content": "The Shoplite shopping cart is designed to provide users with an exceptional purchasing experience, whether they are browsing casually or preparing for a large order. Buyers are allowed to add multiple items from different sellers into a single cart, making it easier to complete transactions without switching between pages. Each item added displays essential details such as product name, price, quantity, seller information, and estimated delivery dates. Users can adjust quantities with a plus or minus button, remove products, or save items for later, allowing for flexible cart management.One of the standout features of Shoplite’s cart is its persistence across sessions. Logged-in users will find that their cart contents remain saved, even if they log out or switch devices. For guest users, the cart is temporarily stored using browser cookies, though checkout requires creating or logging into a Shoplite account. This ensures that important purchase data is not lost.At checkout, users can apply **promo codes, gift cards, or discounts**, which are validated in real time.Moreover, they can collect points based on the amount of monsey they have spent on items and items purchase. These points will allow them to collect future promotions and gifts. The platform also supports split payments across multiple methods for added convenience. The checkout workflow guides buyers through a structured process: confirming shipping addresses, selecting preferred delivery options, and reviewing order summaries before payment is finalized. Delivery preferences, such as express or standard shipping, are clearly displayed along with associated costs.Guest checkout is supported for users who prefer a faster process without creating an account, though certain features like order tracking and saved addresses remain exclusive to registered accounts. For security and transparency, an order confirmation email is sent immediately after payment, summarizing purchase details and providing a reference number. Overall, Shoplite’s cart and checkout system combines usability, flexibility, and reliability to support a smooth shopping journey from product selection to purchase completion.",
        "id": "doc3",
    },
    {
        "title":"Shoplite Payment Methods and Security",
        "content":"Shoplite provides customers with multiple secure payment options to accommodate diverse preferences. Supported methods include major **credit and debit cards, PayPal, and digital wallets** such as Apple Pay,Whish Money, Google Pay. Depending on regional availability, buyers may also have access to local banking methods or installment-based financing. The system is designed for both one-time transactions and stored payment credentials, making future purchases faster and more convenient.Security is the most important criteria during payment processing. Shoplite employs **end-to-end encryption** and complies with PCI-DSS standards to safeguard sensitive financial data. All payment information is tokenized, meaning actual card details are never stored directly on Shoplite’s servers. Users who choose to save their payment methods can rest assured that only encrypted tokens are stored, minimizing exposure to fraud.To further enhance protection, Shoplite’s payment system incorporates automated fraud detection powered by machine learning. This system monitors unusual spending patterns, device inconsistencies, and suspicious login activity. In cases where potential fraud is detected, transactions may be temporarily held for manual review. Users are immediately notified if any irregular activity occurs on their accounts.Shoplite also provides features to keep buyers informed and in control of their payments. Transaction history is available in the account dashboard, and real-time alerts can be enabled via email or SMS. For added account security, customers may activate **two-factor authentication (2FA)**, requiring a secondary verification code before completing high-value purchases. Refunds for returned items are processed securely, with funds typically returned within 5–7 business days to the original payment method.By combining variety, convenience, and robust protection measures, Shoplite ensures that customers can complete transactions confidently, knowing their payment data is safe at every step of the process.",
        "id":"doc4",
    },
    {
        "title": "Shoplite Order Tracking and Delivery",
        "content": "Shoplite provides a transparent and reliable order tracking system to keep buyers informed at every stage of the delivery process. Once an order is placed, the user waits for confirmation. Users receive a detailed confirmation email containing a reference number, product details, and the estimated delivery timeframe. Customers can also access their order history and tracking status directly from their Shoplite account dashboard or mobile app.Order statuses progress through several stages: **Pending Confirmation, Processing, Shipped, Out for Delivery, and Delivered**. Each stage is updated in real time and visible to buyers. For shipped orders, Shoplite integrates with leading logistics providers to offer **real-time tracking links**. Buyers can click on the tracking number to view the carrier’s delivery updates, including location scans and estimated arrival dates. Access to location via google maps or any relevant location based app is required.Delivery options include standard, express, and same-day shipping in select regions. The checkout process allows users to compare delivery methods based on speed and cost. Shoplite also supports delivery scheduling, enabling customers to choose convenient delivery windows where available. Notifications are sent via email, SMS, or in-app alerts when an order is shipped, out for delivery, or successfully delivered.For international orders, estimated delivery times account for customs clearance procedures. Buyers are notified in advance of any customs duties or additional fees that may apply. If delays occur, Shoplite provides proactive updates and escalation channels to customer support.Users can give back a feedback either via the customer care services or online by reviewing the shippement process and order delivery including ponctuality, safety, drivers ethics' , and customer care.Overall, Shoplite’s order tracking and delivery system emphasizes visibility, reliability, and user control. By combining clear status updates, flexible shipping options, and proactive notifications, Shoplite ensures that buyers remain confident and informed throughout the delivery journey.",
        "id": "doc5",
    },
    {
        "title": "Shoplite Return and Refund Policies",
        "content": "Shoplite is committed to ensuring customer satisfaction by offering a **30-day return window** on most eligible products. Buyers can initiate a return through the “My Orders” section of their account, where they select the item, specify a reason for return, and generate a **Return Authorization Number (RAN)**. This number must be included with the returned package to ensure proper processing.Returned items are inspected upon arrival at the seller’s or Shoplite’s fulfillment center. To qualify for a full refund, products must be unused, in their original packaging, and accompanied by proof of purchase. Certain categories such as digital downloads, perishable goods, and final sale items are exempt from return eligibility. For defective or damaged products, buyers may request either a replacement or a refund.Refunds are typically issued to the original payment method within **5–7 business days** after the return is approved. Buyers receive email confirmation once the refund has been processed. If an item was purchased with a promotional code or gift card, the refunded amount reflects the discounted price. Shipping fees are refundable only in cases where Shoplite or the seller is at fault.For added convenience, Shoplite partners with logistics providers to offer **prepaid return shipping labels**, reducing buyer responsibility for return costs in many cases. International customers may face additional return procedures due to customs clearance, and Shoplite provides clear instructions for these scenarios.By maintaining a balance between buyer protection and seller fairness, Shoplite’s return and refund policy ensures transparency and trust. The process is straightforward, designed to minimize disputes, and backed by responsive customer support to handle any exceptions or issues that arise.",
        "id": "doc6",
    },
    {
        "title": "Shoplite Product Reviews and Ratings",
        "content": "Shoplite’s product review and rating system is designed to enhance the user experience by expressing truly and freely about their purchases and help buyers make informed purchasing decisions. After completing a purchase, buyers are invited to leave a review that includes both a star rating (from 1 to 5) and a written feedback. Reviews are tagged with a **“Verified Buyer” badge** when they are submitted by customers who purchased the product directly through Shoplite, ensuring authenticity and reducing the risk of fake feedback.The platform uses detailed reviews by allowing buyers to upload photos or short videos alongside their written comments. These additional materials give prospective buyers a clearer understanding of product quality, size, or performance. Reviews can be filtered by rating, recency, or relevance, allowing shoppers to quickly find the most useful information. Aggregated ratings appear on product pages, displaying the average star rating and total number of reviews.To maintain credibility, Shoplite enforces strict moderation guidelines. Automated systems flag suspicious patterns such as repetitive wording, unusual posting frequency, or reviews originating from the same IP address. Reported reviews undergo human moderation to ensure compliance with Shoplite’s content policies. Sellers are prohibited from offering incentives for positive reviews, and violations may result in account suspension.Buyers who encounter misleading or offensive reviews can use the **“Report Review”** feature, triggering an investigation by Shoplite’s trust and safety team. Sellers may publicly respond to reviews, allowing them to address customer concerns or clarify product details. This creates an open dialogue that can improve trust between buyers and sellers.By balancing openness with moderation, Shoplite’s review system ensures that customers benefit from honest feedback while preventing abuse. As a result, product reviews and ratings play a central role in building confidence, improving transparency, and fostering a trustworthy marketplace environment.",
        "id": "doc7",
    },
    {
        "title": "Shoplite Seller Account Setup and Management",
        "content": "Becoming a seller on Shoplite requires completing a structured **business verification process** to ensure compliance and maintain trust across the marketplace. Sellers begin by visiting the Shoplite seller registration page, where they provide essential business details such as company name, tax identification number, bank account information, and proof of registration. The verification process typically takes **2–3 business days**, during which Shoplite reviews the submitted documentation. Only approved sellers gain access to the full suite of seller tools.Once verified, sellers receive access to a dedicated **Seller Dashboard**, which serves as the control center for managing their online store. The dashboard provides tools for uploading product listings, monitoring inventory, reviewing customer orders, and tracking overall performance metrics. Sellers can customize store branding, add company logos, and create promotional campaigns to attract buyers. Shoplite enforces compliance rules that sellers must follow, including restrictions against counterfeit goods, prohibited items, and misleading advertising. Violations may result in penalties ranging from temporary suspension to permanent account termination. To help sellers succeed, Shoplite offers built-in analytics that track sales trends, customer demographics, and product performance, enabling data-driven decision-making.Sellers are also responsible for maintaining accurate product information and timely shipping practices. Failure to meet minimum performance standards—such as maintaining high ratings or honoring delivery timelines—can negatively impact visibility in search results. Conversely, top-performing sellers may be rewarded with **badges, higher placement in search rankings, and reduced commission rates**.Through a combination of secure onboarding, comprehensive tools, and performance monitoring, Shoplite ensures that sellers can confidently build and grow their businesses while maintaining a trustworthy environment for buyers.",
        "id": "doc8",
    },
    {
        "title": "Shoplite Inventory Management for Sellers",
        "content": "Effective inventory management is critical for sellers to maintain smooth operations and meet customer demand on Shoplite. The **Seller Dashboard** provides multiple options for adding products, including manual entry for individual listings or bulk uploads through CSV files. Bulk uploads are particularly useful for sellers with large catalogs, as they allow for faster updates to product titles, descriptions, images, and pricing. Shoplite also provides a built-in product categorization system to ensure items are properly indexed and discoverable in search results.Sellers benefit from automated **stock alerts**, which notify them when inventory levels fall below a specified threshold. This feature reduces the risk of overselling and allows sellers to restock items before they run out. Shoplite also supports auto-restock settings, enabling sellers to link their inventory systems directly through the Shoplite API for real-time synchronization. This integration ensures that product availability is always accurate, improving buyer trust.Product management tools also allow sellers to set dynamic pricing, schedule promotional discounts, and mark items as “out of stock” when unavailable. Inventory reports within the dashboard display sales velocity, turnover rates, and demand trends. These insights help sellers optimize stock levels and plan ahead for seasonal fluctuations or high-demand events like holidays.Additionally, sellers can manage product variations such as size, color, and style under a single listing, ensuring a more organized shopping experience for buyers. Properly managed inventory not only increases customer satisfaction but also improves seller performance ratings. By combining automation, reporting, and real-time synchronization, Shoplite’s inventory management system empowers sellers to maintain control, reduce errors, and operate more efficiently.",
        "id": "doc9",
    },
    {
        "title": "Shoplite Commission and Fee Structure",
        "content": "Shoplite operates on a transparent **commission-based revenue model** to ensure fairness for both buyers and sellers. The standard commission rate applies to each successful transaction and is deducted automatically from the seller’s payout. This rate varies slightly by product category, with lower commissions for high-volume essentials and higher commissions for premium or luxury items. Clear breakdowns of commission rates are available in the Seller Dashboard.In addition to standard commissions, Shoplite may charge small listing fees for optional premium placements such as featured product spots or homepage highlights. Sellers who choose these promotions gain higher visibility, often resulting in increased sales. However, standard listings without promotions remain free to create. For international transactions, sellers should also account for potential cross-border fees related to currency conversion and additional processing costs.Shoplite offers **Premium Seller Accounts**, which include benefits such as reduced commission percentages, faster payouts, and access to advanced analytics. Premium sellers also receive priority placement in search results and eligibility for exclusive marketing campaigns. The premium plan is subscription-based, with fees billed monthly or annually.Payouts are distributed on a scheduled basis, typically once every **7–14 days**, depending on the seller’s account tier. Funds are transferred directly to the seller’s registered bank account, with detailed invoices available for transparency. Shoplite also provides downloadable tax reports to simplify compliance with local regulations.Importantly, Shoplite enforces a **no hidden fees policy**. All charges are clearly itemized in the seller’s financial dashboard, ensuring there are no unexpected deductions. By combining a predictable commission structure with optional premium services, Shoplite gives sellers the flexibility to choose the plan that best fits their business model while maintaining trust and transparency.",
        "id": "doc10",
    },
    {
        "title": "Shoplite Customer Support and Dispute Resolution",
        "content": "Shoplite prioritizes customer satisfaction by offering a robust **customer support system** that addresses both buyer and seller concerns. Users can access support through multiple channels, including live chat, email, and a comprehensive help center with FAQs and troubleshooting guides. The live chat feature is available 24/7, ensuring that assistance is always accessible regardless of time zones.When issues arise between buyers and sellers—such as undelivered items, defective products, or mismatched descriptions,Shoplite’s **dispute resolution process** comes into play. Buyers can file a dispute directly from their order history, providing evidence such as photos, receipts, or communication logs. Once a dispute is submitted, Shoplite mediates between the buyer and seller to reach a fair resolution. If no agreement is reached, Shoplite’s trust and safety team, which is about professional indviduals ready to solve customer cases with years of experience, makes the final decision, which may result in refunds, partial reimbursements, or replacement products.To streamline this process, Shoplite uses a **tiered escalation system**:1. Buyer contacts seller directly to resolve the issue.2. If unresolved, Shoplite steps in as a mediator.3. Final decisions are enforced by Shoplite, ensuring compliance and fairness.In addition to disputes, customer support also handles account security, recovery, payment issues, and reporting of fraudulent activity. A **“Report an Issue” button** is prominently displayed throughout the platform, enabling quick escalation. For sellers, dedicated account managers may be assigned to high-volume accounts to provide business guidance and handle escalations more efficiently.By combining proactive support with a fair dispute resolution framework, Shoplite builds user trust and ensures that conflicts are handled quickly, transparently, and fairly.",
        "id": "doc11",
    },
    {
        "title": "Shoplite Mobile App Features",
        "content": "Shoplite offers a fully-featured **mobile application** for both iOS and Android, designed to provide users with seamless access to the marketplace anytime, anywhere. The mobile app mirrors the functionality of the web platform while adding mobile-specific features that enhance the shopping experience.Key features include:- **Personalized Recommendations**: Powered by AI-driven algorithms, the app suggests products based on browsing history, purchase patterns, and trending items.- **Push Notifications**: Users receive instant updates on order status, shipping progress, discounts, and live flash sales.- **One-Tap Checkout**: Saved payment methods and addresses enable faster purchases with minimal friction.- **Camera Integration**: Buyers can use visual search by scanning barcodes or uploading product images to find matching or similar items.- **Wishlist Sync**: Wishlists created on mobile automatically sync with the desktop version, ensuring continuity across devices.The app also offers sellers a **mobile dashboard**, allowing them to respond to customer inquiries, adjust pricing, and track inventory on the go. Sellers can enable push alerts for low-stock items or new customer reviews, ensuring they remain responsive even outside of business hours.The possibility to search for a product via a voice speaker is available after the user gives access to the microphone usage within the app.Shoplite ensures that the mobile app remains lightweight, with efficient data usage for regions with slower internet connections. Offline caching allows users to browse previously loaded product pages without an active connection, improving accessibility.With its intuitive design, fast performance, and mobile first features, the Shoplite mobile app strengthens the marketplace’s reach by ensuring that both buyers and sellers remain connected at all times.",
        "id": "doc12",
    },
    {
        "title": "Shoplite Advertising and Promotion Tools",
        "content": "To help sellers mark their presence, grow their visibility and reach more buyers, Shoplite provides a comprehensive suite of **advertising and promotional tools**. Sellers can choose from a list of paid promotion options, including **sponsored product listings**, which display at the top of search results, and **homepage banners** that target high-traffic areas of the platform. These placements operate on a **cost-per-click (CPC)** model, ensuring that sellers only pay when users engage with their promotions.Shoplite also supports **discount campaigns** such as percentage based promotions, buy-one-get-one (BOGO) offers, and limited time flash sales. These campaigns can be scheduled in advance from the Seller Dashboard, making it easier to align promotions with seasonal events or new product launches. Discounted items are highlighted with special badges to capture buyer attention.For more targeted marketing, sellers can use **Shoplite Ads Manager**, a self-service tool that allows them to define budgets, target demographics, and set campaign goals. Advanced analytics provide insights into click-through rates, conversion rates, and return on ad spend (ROAS). This data empowers sellers to optimize campaigns and maximize visibility.In addition to paid advertising, Shoplite rewards **high-performing sellers** with organic promotional boosts. For example, products with high ratings, fast shipping, and strong sales performance are more likely to be featured in trending sections or recommendation feeds.By combining paid promotions with performance based visibility, Shoplite ensures that sellers have multiple pathways to grow their presence, while buyers benefit from discovering relevant and high-quality products.",
        "id": "doc13",
    },
    {
        "title": "Shoplite Security and Privacy Policies",
        "content": "Security and privacy are fundamental to Shoplite’s operations. The platform uses **end-to-end encryption** for all transactions, ensuring that sensitive data such as payment details and personal information remains protected. Two-factor authentication (2FA) is strongly recommended for both buyers and sellers, adding an extra layer of account protection.Shoplite complies with international privacy regulations, including **GDPR** and **CCPA**, to ensure transparent data handling. Users have control over their personal data, with options to download, modify, or delete their information directly from their account settings. A clear privacy policy outlines how data is collected, used, and shared, with strict limits on third-party access.Fraud prevention is powered by **AI-driven monitoring systems** that detect unusual login attempts, suspicious payment activity, and patterns consistent with fraudulent behavior. Transactions flagged as high-risk undergo additional verification steps before processing. Sellers are also required to verify their identities and provide business documentation, reducing the likelihood of fraudulent accounts.To safeguard buyer trust, Shoplite never stores raw credit card data. Instead, it relies on **tokenization and PCI-DSS compliant payment gateways**, ensuring industry-standard financial security. Regular penetration testing and third-party security audits are conducted to identify and patch vulnerabilities.For additional transparency, users are notified of account activity such as logins from new devices or changes to account settings. In case of a data breach, Shoplite has a **rapid-response protocol** that includes immediate user notifications, forced password resets, and cooperation with relevant authorities.By enforcing strict security practices and transparent privacy policies, Shoplite ensures that buyers and sellers can engage in the marketplace with confidence.",
        "id": "doc14",
    },
    {
        "title": "Shoplite Future Expansion and Scalability Plans",
        "content": "Shoplite is designed with scalability in mind, ensuring that the platform can handle rapid growth in both users and transactions. The system architecture leverages **cloud-based infrastructure**, enabling dynamic scaling of servers, storage, and databases as demand increases. This ensures that performance remains stable during peak periods such as holiday sales or large-scale promotional events.Planned expansions include **international market entry**, beginning with regions that have fast-growing e-commerce adoption rates. To support this, Shoplite is working on integrating **multi-currency support, regional payment gateways, and localized languages**. Logistics partnerships with global carriers are also being developed to enable reliable cross-border shipping.On the technology side, Shoplite aims to enhance personalization by investing in **AI-driven recommendation systems** that adapt to user behavior in real time. Future updates also include improved **voice search capabilities**, **augmented reality previews**, and expanded **API integrations** that allow third-party developers to build tools and services around the platform.Scalability also applies to seller tools. Shoplite is exploring advanced inventory forecasting models, predictive analytics for demand planning, and warehouse integration features to help sellers manage growth more efficiently. On the buyer side, loyalty programs and subscription-based benefits are planned to encourage long-term engagement.By combining cloud-native architecture with internationalization, advanced personalization, and continuous innovation, Shoplite is positioned to evolve into a global e-commerce leader while maintaining speed, reliability, and user satisfaction.",
        "id": "doc15",
    },
    {
        "title": "Shoplite Competitor Comparison and Differentiation",
        "content": "In the crowded e-commerce landscape, Shoplite differentiates itself by balancing **buyer trust, seller empowerment, and tech innovation**. While competitors such as Amazon, eBay, and Shopify dominate specific niches, Shoplite offers a hybrid model that incorporates the best aspects of each.- **Amazon Comparison**: Unlike Amazon, which controls its own inventory and prioritizes Prime sellers, Shoplite operates as a neutral marketplace that treats all sellers equally. This levels the playing field for small and medium-sized businesses.- **eBay Comparison**: Whereas eBay is heavily auction-based, Shoplite focuses on fixed-price listings combined with limited-time deals and flash sales. This provides buyers with both affordability and predictability.- **Shopify Comparison**: Shopify enables sellers to create standalone stores but lacks a shared marketplace ecosystem. Shoplite integrates both, offering sellers the tools to manage their storefronts while benefiting from a large shared buyer base.Key differentiators for Shoplite include:- **Transparent Commission Model**: No hidden fees, with clear breakdowns for sellers.- **Trust & Safety Features**: Verified reviews, AI-powered fraud detection, and strict seller onboarding.- **Advanced Buyer Experience**: Personalized recommendations, integrated AR previews, and lightweight mobile design.- **Global Scalability**: Built-in support for localization, cross border logistics, and multi currency payments.By combining fairness, innovation, and accessibility, Shoplite positions itself as a **next-generation marketplace** that supports both small businesses and global buyers, offering a trustworthy alternative to established e-commerce giants.",
        "id": "doc16",
    },
]
print(f"Knowledge base loaded with {len(KNOWLEDGE_BASE)} documents")

Knowledge base loaded with 16 documents


In [ ]:
#Cell 4
PROMPTS={
    "base_retrieval_prompt": {
"role":
"you are a helpful and polite customer service assistant that provides accurate and concise information based on retrieved documents.Adopt a professional, neutral tone suitable for customer-facing support.",
"Goal":
"Provide accurate and honest answers based on the provided Shoplite documents.Prioritize clarity, brevity, and direct citation of source documents.",
"context_guidelines":
["Use the provided information only in the Shoplite documents to answer the user's question."
,"If the information is not available in the documents, respond with 'I'm sorry, I don't have that information.'"
,"Cite the source document(s) used to formulate your answer."
,"Avoid making assumptions or inventing facts or hallucinating or providing information not contained in the documents."],
"response_format":"Answer: [concise answer based strictly on the provided documents]\nSources: [List of document titles referenced, e.g. 'Document 6: Return and Refund Policies']",

                               },
    "role": "You are a precise and efficient customer service assistant for Shoplite who provides brief answers with exact citations from the documents.",
"Goal": "Deliver concise answers that directly address the user's question while strictly citing the relevant document(s) used.",
"context_guidelines":
["Keep answers brief and to the point, ideally under 100 words."
,"Cite only the most relevant document(s) that directly support your answer."
,"Avoid unnecessary elaboration or tangential information."
, "If a claim is not supported, do not state it."],
"response_format":
"Answer: [brief answer based strictly on the provided documents]\nSources: [List of document titles referenced, e.g. 'Document 3: Shoplite Shopping Cart and Checkout Process']",
"confidence_level": "[High/Medium/Low]",
"failure_behavior":
 "If unable to provide a concise answer based on the documents, respond with ""I'm sorry, I don't have that information." "Do not attempt to guess or fabricate information.",

}

In [ ]:
# Cell 5: LLaMA 3.1 8B Instruct with Hugging Face token input
!pip install --quiet transformers accelerate bitsandbytes huggingface_hub

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

# Prompt instructor for their Hugging Face token
hf_token = input("Enter your Hugging Face token: ").strip()
if hf_token:
    login(token=hf_token)

# Model name (gated, requires token + license acceptance)
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Quantization config (to fit Colab GPU memory)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    token=hf_token
)

# Quick smoke test
inputs = tokenizer("User: Hello!\nAssistant:", return_tensors="pt").to(model.device)
out = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))


In [ ]:
# Cell 6: RAG pipeline implementation
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model (small, fast, works well for semantic search)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# --- Step 1: Create embeddings and FAISS index ---
def create_embeddings(docs):
    contents = [doc["content"] for doc in docs]
    embeddings = embedding_model.encode(contents, convert_to_numpy=True)

    # Build FAISS index
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    return index, embeddings

# Build FAISS index once at startup
faiss_index, doc_embeddings = create_embeddings(KNOWLEDGE_BASE)

# --- Step 2: Retrieve top-k documents ---
def retrieve_docs(query, k=3):
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    distances, indices = faiss_index.search(query_embedding, k)
    retrieved = [KNOWLEDGE_BASE[i] for i in indices[0]]
    return retrieved

# --- Step 3: Generate response using structured prompt ---
def generate_response(query, retrieved_docs, prompt_type="base_retrieval_prompt"):
    prompt_cfg = PROMPTS.get(prompt_type, PROMPTS["base_retrieval_prompt"])

    # Build context from retrieved docs
    context_text = "\n\n".join(
        [f"{doc['title']}: {doc['content']}" for doc in retrieved_docs]
    )

    # Construct the full prompt
    full_prompt = (
        f"{prompt_cfg['role']}\n\n"
        f"Goal: {prompt_cfg['goal']}\n\n"
        f"Context:\n{context_text}\n\n"
        f"User question: {query}\n\n"
        f"Instructions:\n{prompt_cfg['context_guidelines']}\n\n"
        f"{prompt_cfg['response_format']}"
    )

    # Encode and generate with the model from Cell 5
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


In [ ]:
# Cell 7: Flask API setup
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "message": "RAG system is running"}), 200

@app.route("/ping", methods=["POST"])
def ping():
    """Direct model call without retrieval"""
    data = request.get_json()
    query = data.get("query", "")

    if not query:
        return jsonify({"error": "No query provided"}), 400

    # Direct model call
    inputs = tokenizer(query, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.7, do_sample=True)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return jsonify({"response": response_text})

@app.route("/chat", methods=["POST"])
def chat():
    """RAG pipeline: retrieve docs + generate structured response"""
    data = request.get_json()
    query = data.get("query", "")
    prompt_type = data.get("prompt_type", "base_retrieval_prompt")

    if not query:
        return jsonify({"error": "No query provided"}), 400

    try:
        # Retrieve relevant documents
        retrieved_docs = retrieve_docs(query, k=3)

        # Generate response with context
        response_text = generate_response(query, retrieved_docs, prompt_type)

        return jsonify({
            "query": query,
            "response": response_text,
            "sources": [doc["title"] for doc in retrieved_docs]
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
# Cell 8: ngrok token input and tunnel creation
from pyngrok import ngrok
import threading

# Ask user (instructor) for their ngrok token at runtime
ngrok_token = input(" Enter your ngrok token: ")
ngrok.set_auth_token(ngrok_token)

# Start Flask app in a background thread
def run_app():
    app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.start()

# Create a public URL with ngrok
public_url = ngrok.connect(5000)
print(f" Public URL: {public_url}")
print(" Flask API is now exposed via ngrok!")


In [ ]:
# Cell 9: Testing and validation
import requests, json

# Paste the public_url from Cell 8 here after ngrok starts
base_url = str(public_url)

if not base_url.startswith("http"):
    raise ValueError(" Please run Cell 8 first to start ngrok and get the public_url.")

print(" Testing API endpoints...")

# 1) Test /health
try:
    r = requests.get(f"{base_url}/health")
    print(" /health:", json.dumps(r.json(), indent=2))
except Exception as e:
    print(" Error calling /health:", str(e))

# 2) Test /ping (direct model call)
try:
    r = requests.post(f"{base_url}/ping", json={"query": "Hello Shoplite"})
    print(" /ping:", json.dumps(r.json(), indent=2))
except Exception as e:
    print(" Error calling /ping:", str(e))

# 3) Test /chat (RAG pipeline)
try:
    r = requests.post(f"{base_url}/chat", json={
        "query": "What is Shoplite and what services does it offer?",
        "prompt_type": "base_retrieval_prompt"
    })
    print(" /chat:", json.dumps(r.json(), indent=2))
except Exception as e:
    print(" Error calling /chat:", str(e))
